### Reset

In [1]:
import pandas as pd
import os
import shutil

# --- KONFIGURATION ---
CSV_FILE = "BEWERTUNGSBOGEN_FINAL.csv"
TEMP_FILE = "BEWERTUNGSBOGEN_TEMP_SAVE.csv"
BACKUP_FILE = "BEWERTUNGSBOGEN_FINAL_OLD_BACKUP.csv"

def reset_ratings():
    # 1. Prüfen ob Datei existiert
    if not os.path.exists(CSV_FILE):
        print(f"FEHLER: Datei {CSV_FILE} nicht gefunden!")
        return

    print("ACHTUNG: Alle bisherigen Bewertungen werden zurückgesetzt!")
    
    # 2. Sicherheits-Backup erstellen
    try:
        shutil.copy(CSV_FILE, BACKUP_FILE)
        print(f"✅ Sicherheitskopie erstellt: {BACKUP_FILE}")
    except Exception as e:
        print(f"Fehler beim Backup: {e}")
        return

    # 3. Daten laden und spalte leeren
    df = pd.read_csv(CSV_FILE)
    rating_col = "Rating (1=Noise, 3=Mix, 5=Precise)"
    
    # Setze die ganze Spalte auf NaN (Leer)
    df[rating_col] = float('nan')
    
    # Speichern
    df.to_csv(CSV_FILE, index=False)
    print(f"✅ Datei {CSV_FILE} wurde bereinigt.")

    # 4. Temp-Datei löschen (WICHTIG!)
    # Sonst lädt das Labeling-Tool beim Starten wieder den alten Stand
    if os.path.exists(TEMP_FILE):
        os.remove(TEMP_FILE)
        print(f"✅ Temporäre Datei {TEMP_FILE} gelöscht.")

# Ausführen
reset_ratings()

ACHTUNG: Alle bisherigen Bewertungen werden zurückgesetzt!
✅ Sicherheitskopie erstellt: BEWERTUNGSBOGEN_FINAL_OLD_BACKUP.csv
✅ Datei BEWERTUNGSBOGEN_FINAL.csv wurde bereinigt.

FERTIG! Du kannst das 'labeling_app_side_by_side' Skript jetzt neu starten.
Es wird bei Bild 1 beginnen.


### Labeling App

In [2]:
import os
from IPython.display import display, HTML, clear_output

# --- KONFIGURATION ---
CSV_FILE = "BEWERTUNGSBOGEN_FINAL.csv"
IMG_DIR = "./blind_study" 
BACKUP_FILE = "BEWERTUNGSBOGEN_TEMP_SAVE.csv"

def labeling_app_side_by_side():
    # 1. Daten laden
    if os.path.exists(BACKUP_FILE):
        df = pd.read_csv(BACKUP_FILE)
    elif os.path.exists(CSV_FILE):
        df = pd.read_csv(CSV_FILE)
    else:
        print("CSV nicht gefunden!")
        return

    rating_col = "Rating (1=Noise, 3=Mix, 5=Precise)"
    
    # Sicherstellen, dass Spalte numerisch ist
    df[rating_col] = pd.to_numeric(df[rating_col], errors='coerce')
    df.loc[df[rating_col] == -1, rating_col] = float('nan')
    
    # Gruppieren nach Image_ID (das Originalbild)
    unfinished_mask = df[rating_col].isna()
    unfinished_ids = df[unfinished_mask]["Image_ID"].unique().tolist()
    
    total_groups = df["Image_ID"].nunique()
    todo_groups = len(unfinished_ids)
    
    if todo_groups == 0:
        print("Alle Bilder fertig bewertet! Glückwunsch.")
        return

    print(f"Start: Noch {todo_groups} Gruppen (Blätter) zu bewerten.")
    
    # 2. Der Loop über die Gruppen
    for i, img_id in enumerate(unfinished_ids):
        # Hole alle 4 Zeilen für dieses Bild (verschiedene Modelle)
        group = df[df["Image_ID"] == img_id]
        
        # HTML String bauen für die Anzeige
        html_str = "<div style='display: flex; flex-direction: row; gap: 10px;'>"
        
        # Liste um die Reihenfolge für die Abfrage zu speichern
        current_batch = []
        
        for idx, row in group.iterrows():
            filename = row["Rating_ID (Filename)"]
            true_class = row["True_Class"]
            img_path = os.path.join(IMG_DIR, filename)
            
            # Prüfen ob Bewertung schon da ist (falls man mittendrin abgebrochen hat)
            current_rating = row[rating_col]
            rating_text = f"Aktuell: {int(current_rating)}" if pd.notna(current_rating) else "Noch offen"
            
           # Fallback für die Anzeige, falls Pfad Probleme macht:
            if os.path.exists(img_path):
                 html_str += f"""
                 <div style='text-align: center; width: 24%; border: 1px solid #eee; padding: 5px;'>
                    <h3 style='margin:0; font-size: 14px;'>Bild {len(current_batch)+1}</h3>
                    <small style='color: #555;'>Ziel: <b>{true_class}</b></small><br>
                    
                    <div style='height: 250px; display: flex; align-items: center; justify-content: center;'>
                        <img src='{img_path}' style='max-width: 100%; max-height: 100%; object-fit: contain;'>
                    </div>
                    
                    <small style='display:block; overflow:hidden; text-overflow:ellipsis; white-space:nowrap;'>{filename}</small>
                    <strong>{rating_text}</strong>
                 </div>
                 """
            else:
                 html_str += f"<div style='width: 24%;'>BILD FEHLT: {filename}</div>"
            
            current_batch.append((idx, filename))
            
        html_str += "</div>"
        
        # --- ANZEIGE ---
        print(f"[{i+1}/{todo_groups}] Bewertung für Blatt ID: {img_id}")
        display(HTML(html_str))
        
        # --- EINGABE LOOP FÜR DIESE 4 BILDER ---
        for pos, (idx, fname) in enumerate(current_batch):
            if pd.isna(df.at[idx, rating_col]):
                valid = False
                while not valid:
                    user_input = input(f"Rating für Bild {pos+1} (ganz links ist 1) [1-5, q=Quit]: ")
                    
                    if user_input.lower() == 'q':
                        print("Gespeichert & Beendet.")
                        df.to_csv(BACKUP_FILE, index=False)
                        df.to_csv(CSV_FILE, index=False)
                        return
                    
                    if user_input in ['1', '2', '3', '4', '5']:
                        df.at[idx, rating_col] = int(user_input)
                        valid = True
                    else:
                        print("Ungültig! Bitte 1-5.")
            
        # Nach der 4er Gruppe speichern und Screen leeren
        df.to_csv(BACKUP_FILE, index=False)
        df.to_csv(CSV_FILE, index=False)
        clear_output(wait=True)

    print("FERTIG! Alle Gruppen bewertet.")
    if os.path.exists(BACKUP_FILE):
        os.remove(BACKUP_FILE)

labeling_app_side_by_side()

FERTIG! Alle Gruppen bewertet.
